**Loading Dataset and creating dataframe**

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from shutil import copy2
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import supervision as sv
import matplotlib.pyplot as plt
from ultralytics import YOLO
from torchvision.ops import box_iou
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
#CHANGE THE FILE PATHS WITH YOUR RESPECTIVE FILE PATH

labeled_images_path = "/kaggle/input/weed-detection-dataset/labeled/images"
labeled_annotations_path = "/kaggle/input/weed-detection-dataset/labeled/annotations"
test_images_path = "/kaggle/input/weed-detection-dataset/test/images"
test_annotations_path = "/kaggle/input/weed-detection-dataset/test/annotations"
unlabeled_images_path = "/kaggle/input/weed-detection-dataset/unlabeled"

In [26]:



def create_dataframe(images_path, annotations_path):
    data = []
    for image_file in os.listdir(images_path):
        image_path = os.path.join(images_path, image_file)
        annotation_file = image_file.replace(".jpg", ".txt")
        annotation_path = os.path.join(annotations_path, annotation_file)
        
        if os.path.exists(annotation_path):
            with open(annotation_path, "r") as f:
                line = f.readline().strip().split()  # Read first line
                if len(line) == 5:  # Ensure valid YOLO format
                    label, x_centre, y_centre, width, height = map(float, line)
                    data.append({"image": image_path, "label": int(label), 
                                 "x_centre": x_centre, "y_centre": y_centre, 
                                 "width": width, "height": height})
    return pd.DataFrame(data)

df = create_dataframe(labeled_images_path, labeled_annotations_path)
df


,image,label,x_centre,y_centre,width,height
0,/kaggle/input/weed-detection-dataset/labeled/i...,1,0.362305,0.416992,0.669922,0.759766
1,/kaggle/input/weed-detection-dataset/labeled/i...,0,0.511719,0.487305,0.953125,0.904297
2,/kaggle/input/weed-detection-dataset/labeled/i...,1,0.437500,0.411133,0.375000,0.341797
3,/kaggle/input/weed-detection-dataset/labeled/i...,1,0.554688,0.445312,0.554688,0.351562
4,/kaggle/input/weed-detection-dataset/labeled/i...,0,0.235352,0.213867,0.087891,0.220703
...,...,...,...,...,...,...
195,/kaggle/input/weed-detection-dataset/labeled/i...,0,0.335938,0.337891,0.632812,0.574219
196,/kaggle/input/weed-detection-dataset/labeled/i...,1,0.499023,0.348633,0.748047,0.603516
197,/kaggle/input/weed-detection-dataset/labeled/i...,1,0.673828,0.279297,0.371094,0.453125
198,/kaggle/input/weed-detection-dataset/labeled/i...,0,0.524414,0.531250,0.767578,0.855469


In [27]:
test_df = create_dataframe(test_images_path, test_annotations_path)

In [28]:
!pip install ultralytics supervision numpy opencv-python torch torchvision


In [30]:
dataset_path = "dataset"
os.makedirs(f"{dataset_path}/images/train", exist_ok=True)
os.makedirs(f"{dataset_path}/images/val", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/val", exist_ok=True)

print("✅ Dataset folders created.")


✅ Dataset folders created.


In [31]:
def move_images(df, img_folder):
    for img_path in df["image"]:
        img_name = os.path.basename(img_path)
        new_path = os.path.join(img_folder, img_name)
        shutil.copy(img_path, new_path)

# ✅ Move training images
move_images(df, "dataset/images/train")

# ✅ Move validation images (from test_df)
move_images(test_df, "dataset/images/val")

print("✅ Images copied to dataset folders.")


✅ Images copied to dataset folders.


In [32]:
def convert_to_yolo(df, label_folder):
    for _, row in df.iterrows():
        img_name = os.path.basename(row["image"]).replace(".jpg", ".txt")  
        label_file = os.path.join(label_folder, img_name)

        with open(label_file, "w") as f:
            f.write(f"{row['label']} {row['x_centre']} {row['y_centre']} {row['width']} {row['height']}\n")

# ✅ Create labels for training and validation
convert_to_yolo(df, "dataset/labels/train")
convert_to_yolo(test_df, "dataset/labels/val")

print("✅ YOLO labels saved.")


✅ YOLO labels saved.


In [33]:
dataset_abs_path = os.path.abspath("dataset")

yaml_content = f"""
path: {dataset_abs_path}
train: images/train
val: images/val
nc: 2
names: ["weed", "non-weed"]
"""

with open("dataset/dataset.yaml", "w") as f:
    f.write(yaml_content)

print("✅ dataset.yaml created.")


✅ dataset.yaml created.


**Training only on labeled dataset**

In [34]:
model = YOLO("yolov8n.pt")  
model.train(data="dataset/dataset.yaml", epochs=20, imgsz=640)

# ✅ Save the trained model
model_path = "yolo_weed_detector.pt"
model.save(model_path)
print(f"✅ YOLO Model trained and saved at {model_path}")


Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

train: Scanning /kaggle/working/dataset/labels/train.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/dataset/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         50         50      0.892      0.897      0.945      0.663



20 epochs completed in 0.018 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.58it/s]


                   all         50         50      0.892      0.897      0.945      0.663
                  weed         24         24      0.905      0.958      0.983      0.769
              non-weed         26         26      0.879      0.836      0.906      0.558


Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/train2
✅ YOLO Model trained and saved at yolo_weed_detector.pt


In [35]:
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="dataset/images/val",
    annotations_directory_path="dataset/labels/val",
    data_yaml_path="dataset/dataset.yaml"
)

In [36]:
def callback(image: np.ndarray) -> sv.Detections:
    result = model(image)[0]  
    return sv.Detections.from_ultralytics(result)


In [60]:
def run_yolo_on_test(test_df, model):
    y_true, y_pred = [], []

    for _, row in test_df.iterrows():
        image_path = row["image"]
        true_label = row["label"]

        results = model(image_path)

        if len(results[0].boxes) == 0:
            y_pred.append(1)  # If no detection, assume it's non-weed
        else:
            best_idx = results[0].boxes.conf.argmax().item()
            best_label = int(results[0].boxes.cls[best_idx].cpu().numpy())
            y_pred.append(best_label)

        y_true.append(true_label)

    return y_true, y_pred

# ✅ Run YOLO and collect predictions
y_true, y_pred = run_yolo_on_test(test_df, model)

# ✅ Compute F1 Score
f1 = f1_score(y_true, y_pred, average="macro")


mean_average_precision = sv.MeanAveragePrecision.benchmark(
    dataset=dataset,
    callback=callback
)

mAP = mean_average_precision.map50_95

final_score = 0.5 * f1 + 0.5 * mAP
print(f"Final Score: {final_score:.4f}")




image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9430_jpeg.rf.55590fd568cfd05cb5a5f38a69781ce9.jpg: 640x640 (no detections), 9.5ms
Speed: 3.0ms preprocess, 9.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9266_jpeg.rf.190539e7297457c48131e14d0eaf9823.jpg: 640x640 (no detections), 7.3ms
Speed: 2.2ms preprocess, 7.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9881_jpeg.rf.ee55ad20cd1edd1f366c6d2641677d66.jpg: 640x640 (no detections), 7.3ms
Speed: 2.2ms preprocess, 7.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9532_jpeg.rf.cec78142a3ad2cbd2466c222131f846a.jpg: 640x640 (no detections), 7.3ms
Speed: 2.1ms preprocess, 7.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/i

**Utilising unlabeled images**

In [41]:
# Load the trained YOLO model
model = YOLO("yolo_weed_detector.pt")
print("✅ YOLO Model loaded.")


✅ YOLO Model loaded.


In [42]:
def generate_labels(image_folder, model):
    """
    Run YOLO on all images in `image_folder` and return a DataFrame with predictions.
    """
    data = []

    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        image = cv2.imread(img_path)

        if image is None:
            print(f"⚠️ Skipping {img_name}, unable to read.")
            continue

        results = model(img_path)

        if len(results[0].boxes) == 0:
            print(f"⚠️ No detections in {img_name}. Assigning default label 1 (non-weed).")
            data.append([img_path, 1, 0, 0, 0, 0])  # Default label for no detection
            continue

        for result in results:
            boxes = result.boxes.xywh.cpu().numpy()  # Convert to YOLO format (x_center, y_center, width, height)
            labels = result.boxes.cls.cpu().numpy()  # Class labels

            for box, label in zip(boxes, labels):
                x_centre, y_centre, width, height = box
                label = int(label)  # Convert label to integer
                data.append([img_path, label, x_centre, y_centre, width, height])

    # Create a DataFrame
    new_df = pd.DataFrame(data, columns=["image", "label", "x_centre", "y_centre", "width", "height"])
    
    return new_df


In [ ]:
# Generate labels for unlabeled images
new_df = generate_labels(unlabeled_images_path, model)

# Display first few rows of generated labels
import ace_tools as tools
tools.display_dataframe_to_user(name="Generated Labels", dataframe=new_df)

print("✅ Labels generated and stored in new_df.")


In [45]:
new_df

,image,label,x_centre,y_centre,width,height
0,/kaggle/input/weed-detection-dataset/unlabeled...,1,256.274933,260.519348,502.361267,363.015961
1,/kaggle/input/weed-detection-dataset/unlabeled...,1,274.809814,252.528229,284.254028,186.668259
2,/kaggle/input/weed-detection-dataset/unlabeled...,0,143.656860,439.980133,37.659081,30.384888
3,/kaggle/input/weed-detection-dataset/unlabeled...,0,93.310394,284.730347,30.548927,29.696014
4,/kaggle/input/weed-detection-dataset/unlabeled...,0,177.002319,98.791946,36.021729,31.420944
...,...,...,...,...,...,...
1139,/kaggle/input/weed-detection-dataset/unlabeled...,0,168.335281,130.319901,302.129913,258.858490
1140,/kaggle/input/weed-detection-dataset/unlabeled...,0,208.441711,241.021637,414.060364,482.043274
1141,/kaggle/input/weed-detection-dataset/unlabeled...,0,294.867249,113.974419,231.541031,227.948837
1142,/kaggle/input/weed-detection-dataset/unlabeled...,0,356.019257,139.217270,164.814453,277.630890


In [46]:
# Normalizing numerical columns to ensure values are between 0 and 1
numerical_columns = ["x_centre", "y_centre", "width", "height"]

# Applying min-max normalization
new_df[numerical_columns] = new_df[numerical_columns].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
new_df

,image,label,x_centre,y_centre,width,height
0,/kaggle/input/weed-detection-dataset/unlabeled...,1,0.555321,0.581240,0.981314,0.713919
1,/kaggle/input/weed-detection-dataset/unlabeled...,1,0.595485,0.563411,0.555263,0.367108
2,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.311290,0.981632,0.073563,0.059756
3,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.202194,0.635257,0.059674,0.058401
4,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.383546,0.220413,0.070365,0.061793
...,...,...,...,...,...,...
1139,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.364765,0.290754,0.590182,0.509080
1140,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.451672,0.537739,0.808827,0.948002
1141,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.638947,0.254286,0.452293,0.448292
1142,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.771457,0.310605,0.321949,0.545998


In [48]:
import pandas as pd

comb_df = pd.concat([new_df, df], ignore_index=True)
comb_df

,image,label,x_centre,y_centre,width,height
0,/kaggle/input/weed-detection-dataset/unlabeled...,1,0.555321,0.581240,0.981314,0.713919
1,/kaggle/input/weed-detection-dataset/unlabeled...,1,0.595485,0.563411,0.555263,0.367108
2,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.311290,0.981632,0.073563,0.059756
3,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.202194,0.635257,0.059674,0.058401
4,/kaggle/input/weed-detection-dataset/unlabeled...,0,0.383546,0.220413,0.070365,0.061793
...,...,...,...,...,...,...
1339,/kaggle/input/weed-detection-dataset/labeled/i...,0,0.335938,0.337891,0.632812,0.574219
1340,/kaggle/input/weed-detection-dataset/labeled/i...,1,0.499023,0.348633,0.748047,0.603516
1341,/kaggle/input/weed-detection-dataset/labeled/i...,1,0.673828,0.279297,0.371094,0.453125
1342,/kaggle/input/weed-detection-dataset/labeled/i...,0,0.524414,0.531250,0.767578,0.855469


In [49]:
dataset_path = "dataset"
os.makedirs(f"{dataset_path}/images/train", exist_ok=True)
os.makedirs(f"{dataset_path}/images/val", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/val", exist_ok=True)

print("✅ Dataset folders created.")


✅ Dataset folders created.


In [50]:
def move_images(df, img_folder):
    for img_path in df["image"]:
        img_name = os.path.basename(img_path)
        new_path = os.path.join(img_folder, img_name)
        shutil.copy(img_path, new_path)

# ✅ Move training images
move_images(comb_df, "dataset/images/train")

# ✅ Move validation images (from test_df)
move_images(test_df, "dataset/images/val")

print("✅ Images copied to dataset folders.")


✅ Images copied to dataset folders.


In [51]:
def convert_to_yolo(df, label_folder):
    for _, row in df.iterrows():
        img_name = os.path.basename(row["image"]).replace(".jpg", ".txt")  
        label_file = os.path.join(label_folder, img_name)

        with open(label_file, "w") as f:
            f.write(f"{row['label']} {row['x_centre']} {row['y_centre']} {row['width']} {row['height']}\n")

# ✅ Create labels for training and validation
convert_to_yolo(df, "dataset/labels/train")
convert_to_yolo(test_df, "dataset/labels/val")

print("✅ YOLO labels saved.")


✅ YOLO labels saved.


In [52]:
dataset_abs_path = os.path.abspath("dataset")

yaml_content = f"""
path: {dataset_abs_path}
train: images/train
val: images/val
nc: 2
names: ["weed", "non-weed"]
"""

with open("dataset/dataset.yaml", "w") as f:
    f.write(yaml_content)

print("✅ dataset.yaml created.")


✅ dataset.yaml created.


**Training on labeled+unlabeled**

In [53]:
model = YOLO("yolov8n.pt")  
model.train(data="dataset/dataset.yaml", epochs=20, imgsz=640)

# ✅ Save the trained model
model_path = "yolo_weed_detector.pt"
model.save(model_path)
print(f"✅ YOLO Model trained and saved at {model_path}")


Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

train: Scanning /kaggle/working/dataset/labels/train... 200 images, 1000 backgrounds, 0 corrupt: 100%|██████████| 1200/1200 [00:00<00:00, 1770.01it/s]


train: New cache created: /kaggle/working/dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/dataset/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         50         50      0.824      0.842        0.9       0.59



20 epochs completed in 0.074 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.10it/s]


                   all         50         50      0.818      0.847      0.899       0.59
                  weed         24         24      0.842      0.886      0.955      0.645
              non-weed         26         26      0.795      0.808      0.843      0.536


Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train3
✅ YOLO Model trained and saved at yolo_weed_detector.pt


In [54]:
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="dataset/images/val",
    annotations_directory_path="dataset/labels/val",
    data_yaml_path="dataset/dataset.yaml"
)

In [55]:
def callback(image: np.ndarray) -> sv.Detections:
    result = model(image)[0]  
    return sv.Detections.from_ultralytics(result)


In [38]:
def run_yolo_on_test(test_df, model):
    y_true, y_pred = [], []

    for _, row in test_df.iterrows():
        image_path = row["image"]
        true_label = row["label"]

        results = model(image_path)

        if len(results[0].boxes) == 0:
            y_pred.append(1)  # If no detection, assume it's non-weed
        else:
            best_idx = results[0].boxes.conf.argmax().item()
            best_label = int(results[0].boxes.cls[best_idx].cpu().numpy())
            y_pred.append(best_label)

        y_true.append(true_label)

    return y_true, y_pred

# ✅ Run YOLO and collect predictions
y_true, y_pred = run_yolo_on_test(test_df, model)

# ✅ Compute F1 Score
f1 = f1_score(y_true, y_pred, average="macro")
print(f"✅ F1 Score: {f1:.4f}")



image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9430_jpeg.rf.55590fd568cfd05cb5a5f38a69781ce9.jpg: 640x640 1 weed, 7.6ms
Speed: 2.3ms preprocess, 7.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9266_jpeg.rf.190539e7297457c48131e14d0eaf9823.jpg: 640x640 1 non-weed, 6.0ms
Speed: 2.1ms preprocess, 6.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9881_jpeg.rf.ee55ad20cd1edd1f366c6d2641677d66.jpg: 640x640 1 non-weed, 6.0ms
Speed: 2.1ms preprocess, 6.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-dataset/test/images/agri_0_9532_jpeg.rf.cec78142a3ad2cbd2466c222131f846a.jpg: 640x640 1 non-weed, 5.9ms
Speed: 2.2ms preprocess, 5.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/weed-detection-data

In [37]:
mean_average_precision = sv.MeanAveragePrecision.benchmark(
    dataset=dataset,
    callback=callback
)

mAP = mean_average_precision.map50_95
print(f"✅ mAP@[0.5:0.95]: {mAP:.4f}")



0: 640x640 2 weeds, 7.4ms
Speed: 2.3ms preprocess, 7.4ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 weed, 7.4ms
Speed: 2.6ms preprocess, 7.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 non-weeds, 7.4ms
Speed: 2.6ms preprocess, 7.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 weed, 7.3ms
Speed: 2.4ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 non-weed, 7.3ms
Speed: 2.4ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 weed, 7.3ms
Speed: 2.3ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 weed, 7.4ms
Speed: 2.3ms preprocess, 7.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 weed, 7.3ms
Speed: 2.3ms preprocess, 7.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 

In [39]:
final_score = 0.5 * f1 + 0.5 * mAP
print(f"Final Score: {final_score:.4f}")


Final Score: 0.8456


As we can see our performance has improved by a lot